In [1]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, TensorDataset
import time
from tqdm import tqdm

/Users/alexzhou/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Information
- We will do a few preliminary exercises and also build a character level MLP language model.
- This model will be similar to the model we did in class, except that we will have characters as tokens, not words.
- You will need a conda environment for this, here is general information on this.
 - https://docs.conda.io/projects/conda/en/latest/user-guide/install/index.html
 - PyTorch: https://anaconda.org/pytorch/pytorch
 
In the code below, FILL-IN the code necessary in the hint string provided.

### Preliminary exercises
- Please fill in the cells below with the asked for data.

In [2]:
torch.manual_seed(1)

In [2]:
e = nn.Embedding(10,5)
v = e(torch.tensor(3))
l = e.weight

In [3]:
l

Parameter containing:
tensor([[ 0.8246, -0.8029, -0.8771,  0.9178, -1.1951],
        [-0.7006,  0.8931,  0.5070,  0.5189,  0.8453],
        [ 0.9370, -0.5061,  0.7507, -0.0704,  0.3276],
        [-0.3582,  0.6241, -0.0508, -1.0284, -0.6014],
        [ 0.5474,  0.1593,  0.6124,  0.1262,  0.9572],
        [ 0.0257,  1.0389, -2.3217,  0.1992,  0.4714],
        [ 0.1319, -1.6183, -0.2496,  1.1969,  1.4502],
        [ 1.3331, -0.1006, -2.0760,  1.1813, -0.6348],
        [ 0.1365, -2.6525, -0.0141, -2.1362,  0.2617],
        [-0.1473,  0.2094, -0.7971,  0.2595,  0.7638]], requires_grad=True)

In [6]:
l.weight

Linear(in_features=10, out_features=5, bias=False)

In [19]:
# Create an embedding layer for a vocabulary of size 10 and the word vectors are each of dimension 5.
e = nn.Embedding(10,5)

# Extract the embedding for the word whose token index is 3. What is the shape of this vector?
v = e(torch.tensor(3))

# Extract the weight matrix from the layer e.
# Create a linear layer (with no bias) of size 10 by 5 and set it's data to the embedding matrix.
l = nn.Linear(5,10, bias = False)
l.weight = e.weight

# Insert inside of the assert below some sort of equality check between l.weight and e.weight; it should pass to true.
# Hint: look up torch.all() and torch.eq()
assert(torch.eq(e.weight, l.weight).all())

In [20]:
# Create a batch of size 2 with entries [0, 1, 2] and [2, 3, 4] in the data batch.
x = torch.tensor([[0, 1, 2], [2, 3, 4]])

In [24]:
# What is the dimesion of this batch ran through the embeding layer?
assert(e(x).shape == torch.Size([2,3,5]))

### Constants and configs used below.

In [55]:
DEVICE = "cpu"
LR = 4.0
BATCH_SIZE = 16
NUM_EPOCHS = 5
MARKER = '.'
# N-gram level; P(w_t | w_{t-1}, ..., w_{t-n+1}).
# We use 3 words to predict the next word.
n = 4
# Hidden layer dimension.
h = 20
# Word embedding dimension.
m = 20

### Get the dataset and the tokenizer.

In [56]:
class CharDataset(Dataset):
    def __init__(self, words, chars):
        self.words = words
        self.chars = chars
        # Inverse dictionaries mapping char tokens to unique ids and the reverse.
        # Tokens in this case are the unique chars we passed in above.
        # Each token should be mappend to a unique integer and MARKER should have token 0.
        # For example, stoi should be like {'.' -> 0, 'a' -> 1, 'b' -> 2} if I pass in chars = '.ab'.
        dic_stoi, dic_itos = {}, {}
        for ele in chars:
            count = 0
            dic_stoi[ele] = count
            dic_itos[count] = ele
            count += 1
        self.stoi = dic_stoi
        self.itos = dic_itos # Inverse mapping.

    def __len__(self):
        # Number of words.
        return len(self.words)

    def contains(self, word):
        # Check if word is in self.words and return True/False if it is, is not.
        return True if word in self.words else False

    def get_vocab_size(self):
        # Return the vocabulary size.
        return len(self.chars)

    def encode(self, word):
        # Express this word as a list of int ids. For example, maybe ".abc" -> [0, 1, 2, 3].
        # This assumes 'a' -> 1, etc.
        result = []
        for char in word:
            result.append(self.stoi[char])
        return result
    
    def decode(self, tokens):
        # For a set of tokens, return back the string.
        # For example, maybe [1, 1, 2] -> "aac"
        result = []
        for tok in tokens:
            result.append(self.dic_itos[tok])
        return result

    def __getitem__(self, idx):
        # This is used so we can loop over the data.
        word = self.words[idx]
        return self.encode(word)

In [64]:
'..'*3

'......'

In [65]:
def create_datasets(window, input_file = 'names.txt'):
    """
    This takes a file of words and separates all the words.
    It then gets all the characters present in the universe of words and then ouputs the statistics. 
    """
    with open(input_file, 'r') as f:
        data = f.read()
    # Split the file by new lines. You should get a list of names.
    words = data.split('\n')
    words = [word.replace(' ', '') for word in words] # This gets rid of any trailing and starting white spaces.
    words = [word for word in words if word] # Filter out all the empty words.

    
    chars = sorted(list(set([char for word in words for char in word]))) # This gets the universe of all characters.
    
    # Will force chars to have MARKER having index 0.
    chars= [MARKER] + chars
    
    # Pad each word with a context window of size n-1.
    # Why? a word like "abc" should becomes "..abc.." if the window is size 3.
    # This is some we can get pair of (x, y) data like this: ".." -> "a", ".a" -> "b", "ab" -> "c", "bc" -> ".", "c." -> "."
    # I.e. this allows us to know that "a" is a start character.
    # So you should get something like ["ab", "c"] -> ["..ab..", "..c.."], for example.
    words = [('.'*(window-1))+word+('.'*(window-1)) for word in words]
            
    print(f"The number of examples in the dataset: {len(words)}")
    print(f"The number of unique characters in the vocabulary: {len(chars)}")
    print(f"The vocabulary we have is: {''.join(chars)}")

    # Partition the input data into a training, validation, and the test set.
    out_of_sample_set_size = min(2000, int(len(words) * 0.1)) # We use 10% of the training set, or up to 2000 examples.
    test_set_size = 1500
    
    # First, get a random permutation of randomly permute of size len(words).
    # Then, convert this to a list. 
    # This index list is used below to get the train, validation, and test sets.
    rp = torch.randperm(len(words)).tolist()
    
    # Get train, validation, and test set.
    train_words = [words[i] for i in rp[:-out_of_sample_set_size]]
    validation_words = [words[i] for i in rp[-out_of_sample_set_size:-test_set_size]]
    test_words = [words[i] for i in rp[-test_set_size:]]    
    
    print(f"We've split up the dataset into {len(train_words)}, {len(validation_words)}, {len(test_words)} training, validation, and test examples")

    # But the data in the data set objects.
    train_dataset = CharDataset(train_words, chars)
    validation_dataset = CharDataset(validation_words, chars)
    test_dataset = CharDataset(test_words, chars)

    return train_dataset, validation_dataset, test_dataset

In [66]:
train_dataset, validation_dataset, test_dataset = create_datasets(n)

The number of examples in the dataset: 32033
The number of unique characters in the vocabulary: 27
The vocabulary we have is: .abcdefghijklmnopqrstuvwxyz
We've split up the dataset into 30033, 500, 1500 training, validation, and test examples


## Explore the data

In [67]:
# Get the first word in "train_dataset"
train_dataset.words[0]

'...eimy...'

In [68]:
# Get the stoi map of train_dataset. How many keys does it have?
print(len(train_dataset.stoi))
print(train_dataset.get_vocab_size())


27
27


### Get the dataloader

In [69]:
def create_dataloader(dataset, window):
    x_list = []
    y_list = []
    # For ech word.
    for i, word in enumerate(dataset):
        # Grab a context of size window and window-1 characters will be in x, 1 will be in y.
        for j, _ in enumerate(word):
            # If there is no widow of size window left, break.
            if j + window > len(word) - 1:
                break
            word_window = word[j:j+window]
            x, y = word_window[:window-1], word_window[-1]
            x_list.append(x)
            y_list.append(y)
            
    return DataLoader(
        TensorDataset(torch.tensor(x_list), torch.tensor(y_list)),
        BATCH_SIZE,
        shuffle=True
    )

In [70]:
train_dataloader = create_dataloader(train_dataset, n)
validation_dataloader = create_dataloader(validation_dataset, n)
test_dataloader = create_dataloader(test_dataset, n)

### Set up the model
- Identical to lecture. Please look over that!

In [33]:
# One of the first Neural language models!
class CharacterNeuralLanguageModel(nn.Module):
    def __init__(self, V, m, h, n):
        super(CharacterNeuralLanguageModel, self).__init__()
        
        # Vocabulary size.
        self.V = " FILL_IN "
        
        # Embedding dimension, per word.
        self.m = " FILL_IN "
        
        # Hidden dimension.
        self.h = " FILL_IN "
        
        # N in "N-gram"
        self.n = " FILL_IN "
        
        # Can you change all this stuff to use nn.Linear?
        # Ca also use nn.Parameter(torch.zeros(V, m)) for self.C but then we need one-hot and this is slow.
        self.C = " FILL_IN " 
        self.H = " FILL_IN "
        self.W = " FILL_IN "
        self.U = " FILL_IN "
        
        self.b = " FILL_IN ")
        self.d = " FILL_IN "
        
        self.init_weights()

    def init_weights(self):
        # Intitialize C, H, W, U in a nice way. Use xavier initialization for the weights.
        # On a first run, just pass.
        " FILL_IN "
        pass # Replace this pass with something else.
        
    def forward(self, x):
        
        # x is of dimenson N = batch size X n-1
        
        # N X (n-1) X m 
        x = " FILL_IN "
        
        # N
        N = " FILL_IN "
        
        # N X (n-1) * m
        x = " FILL_IN "
    
        # N X V
        y = self.b + torch.matmul(x, self.W) + torch.matmul(nn.Tanh()(self.d + torch.matmul(x, self.H)), self.U)
        
        return y

### Set up the model.

In [34]:
# Identical to lecture.
criterion = " FILL_IN "
model = CharacterNeuralLanguageModel(
    train_dataset.get_vocab_size(), m, h, n
).to(DEVICE)
optimizer = " FILL_IN "
scheduler = " FILL_IN "

In [ ]:
# How many parameters does the neural network have?
# Hint: look up model.named_parameters and the method "nelement" on a tensor.
# See also the XOR notebook where we count the gradients that are 0.
# There, we loop over the parameters.
number_parameters = " FILL_IN "

### Train the model.

In [35]:
def calculate_perplexity(total_loss, total_batches):
    return " FILL_IN "

In [44]:
def train(dataloader, model, optimizer, criterion, epoch):
    model.train()
    total_loss, total_batches = 0.0, 0.0
    log_interval = 500

    for idx, (x, y) in tqdm(enumerate(dataloader)):
        optimizer.zero_grad()
        
        logits = model(x)
                        
        # Get the loss.
        loss = " FILL_IN "

        # Do back propagation.
        loss.backward()
                        
        # Clip the gradients so they don't explode. Look at how this is done in lecture.
        " FILL_IN "
        
        # Do an optimization step.
        " FILL_IN "
        
        total_loss += loss.item()
        total_batches += 1
                
        if idx % log_interval == 0 and idx > 0:
            perplexity = calculate_perplexity(total_loss,  total_batches)
            print(
                "| epoch {:3d} "
                "| {:5d}/{:5d} batches "
                "| perplexity {:8.3f} "
                "| loss {:8.3f} "
                .format(
                    epoch,
                    idx,
                    len(dataloader),
                    perplexity,
                    total_loss / total_batches,
                )
            )
            total_loss, total_batches = 0.0, 0

In [45]:
def evaluate(dataloader, model, criterion):
    model.eval()
    total_loss, total_batches = 0.0, 0

    with torch.no_grad():
        for idx, (x, y) in enumerate(dataloader):
            logits = model(x)
            total_loss += criterion(input=logits, target=y.squeeze(-1)).item()
            total_batches += 1
    return total_loss / total_batches, calculate_perplexity(total_loss,  total_batches)

In [46]:
for epoch in range(1, NUM_EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader, model, optimizer, criterion, epoch)
    loss_val, perplexity_val = " FILL_IN "
    scheduler.step()
    print("-" * 59)
    print(
        "| end of epoch {:3d} "
        "| time: {:5.2f}s "
        "| valid perplexity {:8.3f} "
        "| valid loss {:8.3f}".format(
            epoch,
            time.time() - epoch_start_time,
            perplexity_val,
            loss_val
        )
    )
    print("-" * 59)

print("Checking the results of test dataset.")
loss_test, perplexity_test = " FILL_IN "
print("test perplexity {:8.3f} | test loss {:8.3f} ".format(perplexity_test, loss_test))

1020it [00:00, 2664.65it/s]

| epoch   1 |   500/15255 batches | perplexity   10.395 | loss    2.341 
| epoch   1 |  1000/15255 batches | perplexity    9.218 | loss    2.221 


1863it [00:00, 2766.71it/s]

| epoch   1 |  1500/15255 batches | perplexity    8.665 | loss    2.159 
| epoch   1 |  2000/15255 batches | perplexity    8.565 | loss    2.148 


2995it [00:01, 2814.72it/s]

...

| epoch   5 | 14500/15255 batches | perplexity    7.210 | loss    1.975 


15035it [00:05, 2897.91it/s]

| epoch   5 | 15000/15255 batches | perplexity    7.519 | loss    2.017 


15255it [00:05, 2814.68it/s]


-----------------------------------------------------------
| end of epoch   5 | time:  5.45s | valid perplexity    7.565 | valid loss    2.024
-----------------------------------------------------------
Checking the results of test dataset.
test perplexity    7.464 | test loss    2.010 


Hint: For the above, you should see your loss around 2.0 and going down. Similarly to perplexity which should be aroud 7 to 8.

## Generate some text.

In [59]:
def generate_word(model, dataset, window):
    generated_word = []
    # Set the context to a window-1 length array having just the MARKER character's token_id.
    context = " FILL_IN "
    
    while True:
        logits = model(torch.tensor(context).view(1, -1))
        
        # Get the probabilities from the logits.
        # Hint: softmax!
        probs = " FILL_IN "
        
        # Get 1 sample from a multinomial having the above probabilities.
        token_id = torch.multinomial(" FILL_IN ").item()
        
        # Append the token_id to the generated word.
        " FILL_IN "
        
        # Move the context over 1, drop the first (oldest) token and apped the new one above.
        # The size of the resulting context should be the same.
        # For exaple, if it was "[0, 1, 2]" and you generated 4, it should now be [1, 2, 4].
        context = " FILL_IN "
        
        if token_id == 0:
            # If you generate token_id = 0, i.e. '.', break out.
            " FILL_IN "
    # Return and decode the generated word to a string.        
    return ''.join(dataset.decode(generated_word))

In [61]:
torch.manual_seed(1)
for _ in range(50):
    print(generate_word(model, train_dataset, n))

aha.
ele.
lia.
aldi.
jarorsse.
dez.
bemartti.
rielci.
revy.
madlais.
hoanda.
dacelia.
kalaliey.
chis.
mayas.
tya.
jon.
ama.
tze.
karies.
jos.
ahkl.
bamaka.
anyaamiush.
kazerher.
jami.
nnek.
maremellen.
toquyla.
nzygu.
enyl.
yanstram.
ahazoriexsunya.
sermontiroonn.
eifiah.
rosi.
rouivan.
ynn.
ahdityn.
jassavoli.
wun.
jayvarante.
nor.
ilyn.
marri.
allifare.
kalyi.
daslenshanna.
daniellaenimanaililah.
cyle.
